In [ ]:
import socket 

socket.gethostname()

## Labels for Edge Impulse

From here: https://forum.edgeimpulse.com/t/object-detection-dataset-format/4270/3

In [ ]:
import json
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

In [ ]:
bounding_boxes_json = {
    "version": 1,
    "type": "bounding-box-labels",
    "boundingBoxes": {
        
    }
}

dir = '/mnt/c/Users/sabri/Documents/github/thesis/datasets/raw'
dataset = '/mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/auv_increasing_random/train450.txt'

# file_name = ""

with open(dataset, 'r') as file:
    for line in tqdm(file):
        # Get file name and extension
        fn = os.path.basename(line.strip())
        file_name, file_ext = os.path.splitext(fn)

        df = pd.read_csv(dir + '/labels/' + file_name + '.txt', delimiter = " ",  header=None)
        bbox_in = [list(row) for row in df.values]

        ## Get corresponding image size
        image = Image.open(dir + "/images/" + file_name + file_ext)
        width, height = image.size

        ## Convert bounding box annotation to EI format
        bbox_out = {
            file_name + file_ext: []
        }

        for item in bbox_in:

            obj = {
                "label": "",
                "x": 0,
                "y": 0,
                "width": 0,
                "height": 0
                }
            
            label = 'Eusergestes Similis'
            obj["label"] = label
            obj["x"] = int(item[1]*width - item[3]*width/2)
            obj["y"] = int(item[2]*height - item[4]*height/2)
            obj["width"] = int(width * item[3])
            obj["height"] = int(height * item[4])
            
            bbox_out[file_name + file_ext].append(obj)

        bounding_boxes_json["boundingBoxes"].update(bbox_out)

with open("/mnt/c/Users/sabri/Documents/github/thesis/datasets/interim/bounding_boxes.labels", "w") as file:
    json.dump(bounding_boxes_json, file)

In [ ]:
import os
import shutil
import json
import pandas as pd
from tqdm import tqdm
from PIL import Image

# Set paths
raw_dir = '/mnt/c/Users/sabri/Documents/github/thesis/datasets/raw'
interim_dir = '/mnt/c/Users/sabri/Documents/github/thesis/datasets/interim'

train_txt = '/mnt/c/Users/sabri/Documents/github/thesis/datasets/processed/auv/val.txt'

# Set file paths
labels_dir = os.path.join(raw_dir, 'labels')
images_dir = os.path.join(raw_dir, 'images')
output_dir = os.path.join(interim_dir, 'edge_impulse_test')

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Initialize bounding boxes JSON
bounding_boxes_json = {
    "version": 1,
    "type": "bounding-box-labels",
    "boundingBoxes": {}
}

# Loop over train.txt file
with open(train_txt, 'r') as file:
    for line in tqdm(file):
        # Get file name and extension
        filename = os.path.basename(line.strip())
        file_name, file_ext = os.path.splitext(filename)

        # Read label file
        df = pd.read_csv(os.path.join(labels_dir, file_name + '.txt'), delimiter=' ', header=None)
        bbox_in = [list(row) for row in df.values]

        # Get corresponding image size
        image_path = os.path.join(images_dir, filename)
        image = Image.open(image_path)
        width, height = image.size

        # Convert bounding box annotation to EI format
        bbox_out = {
            filename: []
        }
        for item in bbox_in:
            obj = {
                "label": "",
                "x": 0,
                "y": 0,
                "width": 0,
                "height": 0
            }
            label = 'Eusergestes Similis'
            obj["label"] = label
            obj["x"] = int(item[1] * width - item[3] * width / 2)
            obj["y"] = int(item[2] * height - item[4] * height / 2)
            obj["width"] = int(width * item[3])
            obj["height"] = int(height * item[4])
            bbox_out[filename].append(obj)

        # Add bounding box to JSON
        bounding_boxes_json["boundingBoxes"].update(bbox_out)

        # Copy image and label files to output directory
        shutil.copy(image_path, os.path.join(output_dir, filename))
        shutil.copy(os.path.join(labels_dir, file_name + '.txt'), os.path.join(output_dir, file_name + '.txt'))

# Save bounding boxes JSON
with open(os.path.join(output_dir, 'bounding_boxes.labels'), 'w') as file:
    json.dump(bounding_boxes_json, file)


In [ ]:
# Importing the essential libraries
import psutil
import GPUtil
# Testing the psutil library for both CPU and RAM performance details
print(psutil.cpu_percent())
print(psutil.virtual_memory().percent)
# Testing the GPUtil library for both GPU performance details
GPUtil.showUtilization()

In [ ]:
# Importing the required libraries
import psutil
import GPUtil
import numpy as np
import matplotlib.pyplot as plt
# Creating an almost infinite for loop to monitor the details continuously
for i in range(1000):
    # Obtaining all the essential details
    cpu_usage = psutil.cpu_percent()
    mem_usage = psutil.virtual_memory().percent
    print(cpu_usage)
    print(mem_usage)
    # Creating the scatter plot
    plt.scatter(i, cpu_usage, color = "red")
    plt.scatter(i, mem_usage, color = "blue")
    plt.legend(["CPU", "Memory"], loc ="lower right")
    plt.pause(0.05)
    # Obtaining the GPU details
    GPUtil.showUtilization()
# Plotting the information
plt.show()

In [ ]:
import subprocess

# Get overall GPU information
# output = subprocess.check_output(['nvidia-smi'])
# print(output.decode())

# Get GPU utilization
output = subprocess.check_output(['nvidia-smi', '--id=0', '--query-gpu=utilization.gpu', '--format=csv'])
print(output.decode())

# Get GPU memory usage
output = subprocess.check_output(['nvidia-smi', '--id=0', '--query-gpu=memory.used', '--format=csv'])
print(output.decode())

# Get GPU power consumption
output = subprocess.check_output(['nvidia-smi', '--id=0', '--query-gpu=power.draw', '--format=csv'])
print(output.decode())


In [ ]:
from ultralytics import YOLO

path_to_model = '/mnt/c/Users/sabri/Documents/github/thesis/models/auv/auv450.pt'
auv_images = '/mnt/c/Users/sabri/Documents/github/thesis/datasets/raw/images/output*'

model = YOLO(path_to_model)

model.predict(auv_images, save=True, save_conf=True, save_txt=True, imgsz=1280, batch=1, conf=0.5)
